In [7]:
import re
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import col, regexp_replace, trim
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
from datetime import datetime

In [8]:
spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("sales_transactions")\
    .config("spark.eventLog.logBlockUpdates.enabled", True)\
    .getOrCreate()

sc = spark.sparkContext

In [9]:
now = datetime.now()
date_str = now.strftime("%Y%m%d")
hour_str = now.strftime("%H")

In [13]:
input = spark.read.csv(f"hdfs:///data/retail_silver/{date_str}/{hour_str}/sales_transactions_SS_cleaned_{date_str}_{hour_str}.csv", header='true')
input.show(5)

+----------------+----------------+-----------+--------------+--------------+--------------+---------+----------+-----------------+----------------+-------+-------+-------+-------+-------+-----+----------+---------+--------------+----------------+--------------------+-------------+
|transaction_date|  transaction_id|customer_id|customer_fname|cusomter_lname|sales_agent_id|branch_id|product_id|     product_name|product_category|offer_1|offer_2|offer_3|offer_4|offer_5|units|unit_price|is_online|payment_method|shipping_address|      customer_email|discount_perc|
+----------------+----------------+-----------+--------------+--------------+--------------+---------+----------+-----------------+----------------+-------+-------+-------+-------+-------+-----+----------+---------+--------------+----------------+--------------------+-------------+
|      2022-12-13|trx-675189967400|      85552|         James|         Smith|             8|        3|        20|            Heels|        Footwear|   

In [ ]:
customers = input.select("customer_id","customer_fname","cusomter_lname","customer_email").dropDuplicates()
customers